In [45]:
import xml
import xml.etree.ElementTree as ET
from os import listdir
from os.path import isfile, join
import os
import re
from collections import defaultdict

In [8]:
os.getcwd()

'/home1/zishan/raghav/notebooks'

In [63]:
mypath='/home1/zishan/raghav/Data/Corrected/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [64]:
onlyfiles[:10]

['navbharat_2009_3_6_4231845.xml',
 'in-drought-hit-latur-10000-litres-of-water-for-bjp-minister-helipad2.txt.xml',
 'blast-at-fruit-market-in-northeastern-nigerias-yola-kills-3216.txt.xml',
 'iraq-suicide-bomber-kills-at-least-60-at-checkpoint8.txt.xml',
 'firozpur-tornado-in-ferozpur-heavy-loss-of-property-175094524.txt.xml',
 'navbharat_2003_8_30_57158709final',
 'navbharat_2009_1_3_3927424.xml',
 'navbharat_2004_9_27_865156',
 'new-delhi4.txt.xml',
 'navbharat_2004_4_20_627443final']

In [122]:
print(len(onlyfiles))
print(len(set(onlyfiles)))

101
101


In [145]:
import xml.etree.ElementTree as etree

for filename in onlyfiles:
    tree = etree.parse('/home1/zishan/raghav/Data/Corrected/'+str(filename)).getroot()

    results = []

    w_tags = []
    for element in tree.findall('P/*'):
        if element.tag == 'W':
            w_tags.append(element.text)

        for search_element in element.findall('W'):
            w_tags.append(search_element.text)

        if re.match('EMOTION\d-SENTENCE' ,element.tag):
            dict_to_get_class = defaultdict(list)
            for child in element.iter():
                if child.tag=='W':
                    w_tags.append(child.text)
                elif re.match('EMOTION\d-SENTENCE', child.tag):
                    id_sentence = child.tag[7]
                    dict_to_get_class[id_sentence].append(child.attrib['TYPE'])
                elif re.match('EMOTION\d-INTENSITY-SENTENCE', child.tag):
                    id_sentence = child.tag[7]
                    dict_to_get_class[id_sentence].append(float(child.attrib['TYPE']))
            try:
                temp = sorted(dict_to_get_class.items(), key= lambda e: e[1][1], reverse=True)
                label = temp[0][1][0]
                w_tags.append([label])
                results.append(w_tags)
                w_tags = []
            except Exception as e:
                print('exception in file %s' %(str(filename)))
                print(''.join(w_tags))
                print('*****************')
                w_tags = []


    for sentence in results:
        label = sentence[-1]
        words = sentence[:-1]
        words = [word.strip(' ') for word in words]
        with open('/home1/zishan/raghav/Data/data.txt','a+') as f:
            string = ' '.join(words) + '\t' + label[0]+'\n'
            f.write(string)


 

exception in file blast-at-fruit-market-in-northeastern-nigerias-yola-kills-3216.txt.xml
 हाल  ही  अक्टूबर  में  योला  में  एक  नवनिर्मित  मस्जिद  में  भी  धमाका  हुआ  था,  जिसमें  55  लोगों  की  जान  चली  गई  थी  और  100  से  अधिक  घायल  हो  गए  थे। 
*****************
exception in file firozpur-tornado-in-ferozpur-heavy-loss-of-property-175094524.txt.xml
 200  किमी  से  अधिक  की  रफ्तार  की 
*****************
exception in file navbharat_2004_10_21_894664
 चक्रवात  के  पूर्व  की  ओर  प्रशांत  महासागर  की  तरफ  बढ़ने  से  पहले  कल  इसने  पूरे  जापान  को  प्रभावित  किया।  किया। 
*****************
exception in file navbharat_2004_9_8_843167
 भयंकर  तूफान  से  ग्रस्त  कैरीबिआई  द्वीप  ग्रेनेडा  में  रह  रहे  भारतीय  पूरी  तरह  से  सुरक्षित  हैं  और  स्थिति  को  बेहतर  बनाने  का  प्रयास  कर  रहे  हैं। 
*****************
exception in file chhattisgarh5.txt.xml
 हाल  ही  अक्टूबर  में  योला  में  एक  नवनिर्मित  मस्जिद  में  भी  धमाका  हुआ  था,  जिसमें  55  लोगों  की  जान  चली  गई  थी  और  100 